In [ ]:
# Install PrimeHub SDK.
!pip install primehub-python-sdk==0.3.5

In [ ]:
# Please filled in the PrimeHub cluster.
PRIMEHUB_CLUSTER = <primehub-cluster-url>

In [ ]:
# Please filled in the kaggle config.
KAGGLE_CONFIG = <kaggle-config-url>

In [ ]:
# Setting the primehub sdk config.
from primehub import PrimeHub, PrimeHubConfig
ph = PrimeHub(PrimeHubConfig())

In [ ]:
# Login config setting
import os
if not os.path.exists("/root/.primehub/config.json"):
    ph.config.generate(PRIMEHUB_CLUSTER)

In [ ]:
# Get config json file information.
!cat /root/.primehub/config.json

In [ ]:
# Setting the primehub sdk config.
ph = PrimeHub(PrimeHubConfig())

In [ ]:
# Check the SDK setup is ready.
if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("PrimeHub Python SDK couldn't get the group information, please check the configuration.")

In [ ]:
primehub_job_config = {
    "instanceType": "cpu-1",
    "image": "tf-2",
    "displayName": "credit-card-model-training",
    "command": """git clone https://github.com/InfuseAI/showcase.git

pip install kaggle
export KAGGLE_USERNAME={}
export KAGGLE_KEY={}

cd ~/credit-card-recommendation/data/original
kaggle datasets download -d sajidhussain3/jobathon-may-2021-credit-card-lead-prediction
unzip jobathon-may-2021-credit-card-lead-prediction.zip

cd ~/credit-card-recommendation/
pip install -r requirements.txt
python main.py
""".format(KAGGLE_CONFIG['username'], KAGGLE_CONFIG['key'])
}

In [ ]:
# Submit a job to train model.
short_job = ph.jobs.submit(primehub_job_config)
print(short_job)

# Wait the job to be done
print('[ Waiting ]')
ph.jobs.wait(short_job['id'])
print('[ Job Done ]')

# Get logs
logs = ph.jobs.logs(short_job['id'])
print('[ Job Logs ]')
for l in logs:
    print(l)

In [ ]:
# List all versions of a model
model_information = []
model_versions = ph.models.list_versions('credit-card-recommendation')
for version in model_versions:
    model_information.append(version)

In [ ]:
from primehub.utils import ResourceNotFoundException

# Create a deployment
deployment_id=f"credit-card-recommendation" 
config = {
    "id": deployment_id,
    "name": f"Credit card recommendation",
    "modelImage": "infuseai/sklearn-prepackaged:v0.1.0",
    "modelURI": "models:/credit-card-recommendation/{}".format(model_information[-1]['version']),
    "instanceType": "cpu-1",
    "replicas": 1
}

# Delete a deployment
try:
    ph.deployments.delete(deployment_id)
    print("Finish delete model deployment")
except ResourceNotFoundException as e:
    print("Model deployment resource not found.")

# Create a deployment
deployment = ph.deployments.create(config)
print(f"""
Deployment Id: {deployment_id} 
Deployment Page: {PRIMEHUB_CLUSTER}/console/g/phusers/deployments/{deployment_id}
""")

# Wait for the deployment ready
print('[ Waiting ]')
ph.deployments.wait(deployment['id'])
print('[ Deployment Ready ]')
deployment = ph.deployments.get(deployment['id'])

In [ ]:
# prediction_columns = ['Gender', 'Age', 'Region_Code', 'Vintage', 'Avg_Account_Balance',
#       'Is_Active', 'Occupation_0', 'Occupation_1', 'Occupation_2',
#       'Occupation_3', 'Channel_Code_0', 'Channel_Code_1', 'Channel_Code_2',
#       'Channel_Code_3', 'Credit_Product_0', 'Credit_Product_1',
#       'Credit_Product_2']

In [ ]:
# Use model server to predict the test data
import json
import requests

url = deployment['endpoint']
data = {
    "data": {
        "ndarray": [[ 0, 2.57229854, 1, 0.12489877, -1.18575425, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]]
    }
}
headers = {'Content-type': 'application/json'}

result = requests.post(url, data=json.dumps(data), headers=headers)
if result.status_code != 200:
  print(f"{result.status_code}: {result.reason}")

predict_result = json.loads(result.content.decode())

print(predict_result)